# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [118]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
import sys
sys.path.append('..')
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
path = "../WeatherPy/weather_df.csv"
weather_df = pd.read_csv(path)
weather_df[:4]

,Unnamed: 0,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Description,Wind_Speed,Country,Date
0,0,mandalgovi,45.76,106.27,4.68,58,0,clear sky,8.70,MN,2020-05-10 02:25:48
1,1,hilo,19.73,-155.09,27.00,57,90,overcast clouds,2.60,US,2020-05-10 02:22:00
2,2,busselton,-33.65,115.33,16.67,45,40,scattered clouds,0.93,AU,2020-05-10 02:25:49
3,3,bredasdorp,-34.53,20.04,14.00,93,0,clear sky,1.00,ZA,2020-05-10 02:25:49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
len(weather_df)
len(weather_df.City)

535

In [17]:
#Set up data
locations = weather_df[['Latitude','Longitude']]
weights = weather_df['Humidity']

citydictlist = []
for row in range(len(weather_df)):
    dict = {
        'name': weather_df['City'][row],
        'location': (weather_df['Latitude'][row], 
                     weather_df['Longitude'][row]),
        'max_temp': weather_df['Max_Temp'][row],
        'humidity': weather_df['Humidity'][row],
        'cloudiness':weather_df['Cloudiness'][row],
        'description':weather_df['Description'][row],
        'wind_speed':weather_df['Wind_Speed'][row],
        'country':weather_df['Country'][row],
        'date':weather_df['Date'][row]
    }
    citydictlist.append(dict)

In [35]:
#set up for mapping
gmaps.configure(api_key=g_key)
map_types = {'ROADMAP': 'default Google Maps style',
             'SATELLITE': 'simple satellite view', 
             'HYBRID': 'satellite view with roads and cities overlaid', 
             'TERRAIN': 'emphasizes terrain features'}

figure_layouts = {
    'width': 'figure width in px ("400px") or %. Default: 100%. Other CSS formats allowed, like "em"',
    'height': 'figure height in px or %; default = "420px". Other CSS forms allowed.',
    'border': 'must be valid CSS border',
    'padding': 'must be valid CSS padding, eg "2px" or ("1px 2px 1px 2px"). default = 0',
    'margin': 'gap between border and figure container, valid CSS margin, default = 0'
}

#marker_locations = [(0, 0), (32, 140)]
#markers = gmaps.marker_layer(marker_locations)

#dissipating = True

In [72]:
#base map configuration
centermap_coordinates = (0, -10)
map_type = 'HYBRID'
width = '600px'
height = '450px'
border = '2px dotted medium dashed green'
padding = '3px'
zoom_level = 1.22

figure_layout = {
    'width': width,
    'height': height,
    'border': border,
    'padding': padding
}

In [80]:
fig = gmaps.figure(map_type = map_type, 
                   center = centermap_coordinates,
                   zoom_level = zoom_level, 
                   layout = figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, 
                                  weights= weights))
gmaps.heatmap_layer.point_radius = 0.35
gmaps.heatmap_layer.max_intensity = None
fig

Figure(layout=FigureLayout(border='2px dotted medium dashed green', height='450px', padding='3px', width='600p…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [89]:
weather_df['Max_Temp_Fahr'] = weather_df['Max_Temp'] * 9/5 + 32 
weather_df['Wind_Speed_mph'] = weather_df['Wind_Speed'] *  2.237

weather_df[:4]

,Unnamed: 0,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Description,Wind_Speed,Country,Date,Max_Temp_Fahr,Wind_Speed_mph
0,0,mandalgovi,45.76,106.27,4.68,58,0,clear sky,8.70,MN,2020-05-10 02:25:48,40.424,19.46190
1,1,hilo,19.73,-155.09,27.00,57,90,overcast clouds,2.60,US,2020-05-10 02:22:00,80.600,5.81620
2,2,busselton,-33.65,115.33,16.67,45,40,scattered clouds,0.93,AU,2020-05-10 02:25:49,62.006,2.08041
3,3,bredasdorp,-34.53,20.04,14.00,93,0,clear sky,1.00,ZA,2020-05-10 02:25:49,57.200,2.23700


In [ ]:
'''Original Criteria for May: 
-- temperatures below 85
-- temperatures above 31
-- Humidity below 82
-- 15 mph wind speeds or less
'''

In [107]:
df = weather_df
laura_cityframe = df[(df.Wind_Speed_mph <=8)&
                     (df.Max_Temp_Fahr < 71)&
                     (df.Max_Temp_Fahr>62) &
                     (df.Cloudiness < 45)&
                     (df.Humidity < 60)]
len(laura_cityframe)
'''Final Criteria:
- wind speed <= 8 mph
- temp >62 and <71
- less than 45% cloudy
- less than 60% humidity'''

15

In [108]:
laura_cityframe

,Unnamed: 0,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Description,Wind_Speed,Country,Date,Max_Temp_Fahr,Wind_Speed_mph
2,2,busselton,-33.65,115.33,16.67,45,40,scattered clouds,0.93,AU,2020-05-10 02:25:49,62.006,2.08041
25,25,east london,-33.02,27.91,18.23,55,1,clear sky,2.62,ZA,2020-05-10 02:22:48,64.814,5.86094
100,100,charyshskoye,51.40,83.56,19.25,40,0,clear sky,1.87,RU,2020-05-10 02:27:08,66.650,4.18319
248,248,jalu,29.03,21.55,18.45,26,0,clear sky,3.14,LY,2020-05-10 02:30:09,65.210,7.02418
318,318,hami,42.80,93.45,18.84,23,0,clear sky,1.14,CN,2020-05-10 02:31:56,65.912,2.55018
341,341,norman,35.22,-97.44,17.22,55,1,clear sky,3.10,US,2020-05-10 02:27:50,62.996,6.93470
358,358,altay,47.87,88.12,16.70,27,0,clear sky,3.18,CN,2020-05-10 02:32:12,62.060,7.11366
378,378,talakan,50.32,130.37,19.56,36,0,clear sky,2.91,RU,2020-05-10 02:32:59,67.208,6.50967
382,382,constitucion,23.99,-104.67,18.44,41,28,scattered clouds,2.61,MX,2020-05-10 02:33:00,65.192,5.83857
400,400,sakakah,29.97,40.21,17.00,48,0,clear sky,2.10,SA,2020-05-10 02:33:09,62.600,4.69770


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [125]:
laura_cityframe["Hotel_Name"] = ""
laura_cityframe.head()

/Users/lauranicholson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Description,Wind_Speed,Country,Date,Max_Temp_Fahr,Wind_Speed_mph,Hotel_Name
2,None,busselton,-33.65,115.33,16.67,45,40,scattered clouds,0.93,AU,2020-05-10 02:25:49,62.006,2.08041,
25,None,east london,-33.02,27.91,18.23,55,1,clear sky,2.62,ZA,2020-05-10 02:22:48,64.814,5.86094,
100,None,charyshskoye,51.40,83.56,19.25,40,0,clear sky,1.87,RU,2020-05-10 02:27:08,66.650,4.18319,
248,None,jalu,29.03,21.55,18.45,26,0,clear sky,3.14,LY,2020-05-10 02:30:09,65.210,7.02418,
318,None,hami,42.80,93.45,18.84,23,0,clear sky,1.14,CN,2020-05-10 02:31:56,65.912,2.55018,


In [169]:
df = laura_cityframe

df = laura_cityframe.reset_index()


In [170]:
#parameters
#

baseurl ='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
radius = 'radius=5000'
key = '&key=YOUR_API_KEY'
type = 'type=lodging'

hoteldict = {}
hoteldictlist = []

for row in range(len(df)-1):
    lat = df['Latitude'][row]
    lng= df['Longitude'][row]
    location = f'location={lat},{lng}'
    params = f'{location}&{radius}&{type}&key={g_key}'
    url=f'{baseurl}{params}'
    r = requests.get(url).json()
    try:
        print(r['results'][0]['name'])
        df['Hotel_Name'] = r['results'][0]['name']
        hoteldict['Name'] = r['results'][0]['name']
        hoteldict['Location'] = (r['results'][0]['geometry']['location']['lat'],
                                 r['results'][0]['geometry']['location']['lng'])
    except:
        df['Hotel_Name'] = 'No Lodging Found'
        hoteldict['Name'] = 'No Lodging Found'
        hoteldict['Location'] = (lat, lng) 
    hoteldict['City'] = df['City'][row]
    hoteldict['Country'] = df['Country'][row]
    hoteldictlist.append(hoteldict)
    hoteldict={}

pprint(hoteldictlist)

Observatory Guest House
Tu Casa
Zelenyy Dom Gornyy Charysh
Jalu Hotel
Laoyutang Resort
Country Inn & Suites by Radisson, Norman, OK
Jinqiao Hotel
Hotel Posada San Agustin
Fakhamat Al Orjoana Apartment
Delong Hotel
Hotel Provincial
Jinjiang Hotel
[{'City': 'busselton',
  'Country': 'AU',
  'Location': (-33.64639289999999, 115.3486762),
  'Name': 'Observatory Guest House'},
 {'City': 'east london',
  'Country': 'ZA',
  'Location': (-32.9851587, 27.9235698),
  'Name': 'Tu Casa'},
 {'City': 'charyshskoye',
  'Country': 'RU',
  'Location': (51.3911776, 83.55750900000001),
  'Name': 'Zelenyy Dom Gornyy Charysh'},
 {'City': 'jalu',
  'Country': 'LY',
  'Location': (29.03272369999999, 21.5458054),
  'Name': 'Jalu Hotel'},
 {'City': 'hami',
  'Country': 'CN',
  'Location': (42.802482, 93.491396),
  'Name': 'Laoyutang Resort'},
 {'City': 'norman',
  'Country': 'US',
  'Location': (35.2064605, -97.4839878),
  'Name': 'Country Inn & Suites by Radisson, Norman, OK'},
 {'City': 'altay',
  'Country':

In [151]:


#pprint(r)
df['City'][0]
#r['results'][0]['name']
#we'll assume all hotels are IN the city proper

'busselton'

In [197]:
#Set up data
locations = weather_df[['Latitude','Longitude']]
weights = weather_df['Humidity']


# Add the layer to the map
#fig.add_layer(markers)

In [198]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**item) for item in hoteldictlist]
hotellocations = [hotel['Location'] for hotel in hoteldictlist]



In [199]:
#base map configuration
centermap_coordinates = (0, -10)
map_type = 'HYBRID'
width = '600px'
height = '450px'
border = '2px dotted medium dashed green'
padding = '3px'
zoom_level = 1.22

figure_layout = {
    'width': width,
    'height': height,
    'border': border,
    'padding': padding
}

In [200]:
fig = gmaps.figure(map_type = map_type, 
                   center = centermap_coordinates,
                   zoom_level = zoom_level, 
                   layout = figure_layout)

fig.add_layer(gmaps.heatmap_layer(locations, 
                                  weights= weights))

gmaps.heatmap_layer.point_radius = 0.35
gmaps.heatmap_layer.max_intensity = None

marker_layer = gmaps.marker_layer(hotellocations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

#gmaps.marker_layer(locations5g)
# Add the layer to the map
#fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='2px dotted medium dashed green', height='450px', padding='3px', width='600p…

In [298]:
#Set up data
locations = df[['Latitude','Longitude']]
weights = df['Humidity']
nohoteldictlist = [hoteldict for hoteldict in hoteldictlist if hoteldict['Name'] == 'No Lodging Found']
yeshoteldictlist = [hoteldict for hoteldict in hoteldictlist if hoteldict['Name'] != 'No Lodging Found']


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**item) for item in yeshoteldictlist]
hotellocations = [hotel['Location'] for hotel in yeshoteldictlist]
nohotel_info = [info_box_template.format(**item) for item in nohoteldictlist]
nohotellocations = [hotel['Location'] for hotel in nohoteldictlist]

#base map configuration
centermap_coordinates = (0, 18)
map_type = 'HYBRID'
width = '600px'
height = '450px'
border = '2px dotted medium dashed green'
padding = '3px'
zoom_level = 1.22

figure_layout = {
    'width': width,
    'height': height,
    'border': border,
    'padding': padding
}

fig = gmaps.figure(map_type = map_type, 
                   center = centermap_coordinates,
                   zoom_level = zoom_level, 
                   layout = figure_layout)

fig.add_layer(gmaps.heatmap_layer(locations, 
                                  weights= weights))

gmaps.heatmap_layer.point_radius = 0.35
gmaps.heatmap_layer.max_intensity = 100

marker_layer = gmaps.marker_layer(hotellocations, info_box_content=hotel_info)
nohotel_symbols = gmaps.symbol_layer(nohotellocations, info_box_content=nohotel_info, 
                                     fill_color='rgba(50, 70, 200, 1)',
                                     stroke_color='rgba(10, 50, 100, 1)', scale=3)

fig.add_layer(marker_layer)
fig.add_layer(nohotel_symbols)

#gmaps.marker_layer(locations5g)
# Add the layer to the map
#fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='2px dotted medium dashed green', height='450px', padding='3px', width='600p…

In [296]:
#testing gmaps out


locations = weather_df[['Latitude','Longitude']]
#weights = weather_df['Max_Temp_Fahr'] 

for temp in weather_df['Max_Temp_Fahr']:
    if temp <40:
        weather_df['weight']= 0.25
    elif temp <50:
        weather_df['weight'] = 0.25
    elif temp <60:
        weather_df['weight'] = 0.5
    elif temp <75:
        weather_df['weight'] = 0.5
    elif temp <80:
        weather_df['weight'] = 0.75
    elif temp < 90:
        weather_df['weight'] = 0.85
    elif temp >=90:
        weather_df['weight'] = 1
weights = weather_df['weight'] 
#base map configuration
centermap_coordinates = (5, -10)
map_type = 'HYBRID'
width = '600px'
height = '450px'
border = '2px dotted medium dashed green'
padding = '3px'
#zoom_level = 1.22
zoom_level = 2
figure_layout = {
    'width': width,
    'height': height,
    'border': border,
    'padding': padding
}

#heatmap_layer.point_radius = 8
#heatmap_layer.max_intensity = 100
'''
heatmap.gradient = ['white', 
                    'silver', 
                    'gray']
'''
heatmap_gradient = [
    (10, 10, 250, 0),
    #(10, 10, 250, 1),
    (10, 100, 250, 1),
    #(10, 250, 250, 1),
   # (100, 100, 200, 1),
    #(100, 200, 200, 1),
    #(240, 240, 0, 1),
    (255, 50, 50, .75)
]
#heatmap.opacity = 0.5





fig = gmaps.figure(map_type = map_type, 
                   center = centermap_coordinates,
                   zoom_level = zoom_level, 
                   layout = figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, 
                                  weights= weights, dissipating=False, 
                                  max_intensity=0.99, 
                                  point_radius = 7, gradient = heatmap_gradient))

fig

Figure(layout=FigureLayout(border='2px dotted medium dashed green', height='450px', padding='3px', width='600p…

In [ ]:

m = gmaps.Map()
heatmap_layer = gmaps.WeightedHeatmap(data=tritupledata,
                             )
m.add_layer(heatmap_layer)



In [297]:
max(weather_df['Humidity'])

100

In [168]:
hoteldictlist

[{'Name': 'Observatory Guest House',
  'Location': (-33.64639289999999, 115.3486762),
  'City': 'busselton',
  'Country': 'AU'},
 {'Name': 'Tu Casa',
  'Location': (-32.9851587, 27.9235698),
  'City': 'east london',
  'Country': 'ZA'},
 {'Name': 'Zelenyy Dom Gornyy Charysh',
  'Location': (51.3911776, 83.55750900000001),
  'City': 'charyshskoye',
  'Country': 'RU'},
 {'Name': 'Jalu Hotel',
  'Location': (29.03272369999999, 21.5458054),
  'City': 'jalu',
  'Country': 'LY'},
 {'Name': 'Laoyutang Resort',
  'Location': (42.802482, 93.491396),
  'City': 'hami',
  'Country': 'CN'},
 {'Name': 'Country Inn & Suites by Radisson, Norman, OK',
  'Location': (35.2064605, -97.4839878),
  'City': 'norman',
  'Country': 'US'},
 {'Name': 'Jinqiao Hotel',
  'Location': (47.84586230000001, 88.1381018),
  'City': 'altay',
  'Country': 'CN'},
 {'Name': 'Hotel Posada San Agustin',
  'Location': (24.024374, -104.673017),
  'City': 'constitucion',
  'Country': 'MX'},
 {'Name': 'Fakhamat Al Orjoana Apartment

In [ ]:
fig = gmaps.figure()
#heatmap_layer = gmaps.heatmap_layer(locations)
heatmap_layer = gmaps.heatmap_layer(locations, weights='Humidity', 
                                dissipating=False, max_intensity=3,
                                point_radius=0.35)
heatmap_layer2 = gmaps.heatmap_layer(locations5g)
fig.add_layer(heatmap_layer)

In [ ]:
# Add marker layer ontop of heat map


# Display figure


## Appendix and notes

In [ ]:

fig = gmaps.figure()
#heatmap_layer = gmaps.heatmap_layer(locations)
heatmap_layer = gmaps.heatmap_layer(locations, weights='Humidity', 
                                dissipating=False, max_intensity=3,
                                point_radius=0.35)
heatmap_layer2 = gmaps.heatmap_layer(locations5g)
fig.add_layer(heatmap_layer)
#markers = gmaps.marker_layer(locations5g)
# Add the layer to the map
#fig.add_layer(markers)
#embed_minimal_html('celltowermap_final.html', views=[fig])
fig

## Pop up boxes to markers
nuclear_power_plants = [
    {'name': 'Atucha', 'location': (-34.0, -59.167), 'active_reactors': 1},
    {'name': 'Embalse', 'location': (-32.2333, -64.4333), 'active_reactors': 1},
    {'name': 'Armenia', 'location': (40.167, 44.133), 'active_reactors': 1}
]

plant_locations = [plant['location'] for plant in nuclear_power_plants]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Number reactors</dt><dd>{active_reactors}</dd>
</dl>
"""
plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]

marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

#If you need to draw more complex shape on maps, use the symbol_layer function. Symbols represent each latitude, longitude pair with a circle whose colour and size you can customize.

starbucks_layer = gmaps.symbol_layer(
    starbucks_df, fill_color='green', stroke_color='green', scale=2
)
starbucks_layer = gmaps.symbol_layer(
    starbucks_df, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 150, 0, 0.4)', scale=2
)

kfc_layer = gmaps.symbol_layer(
    kfc_df, fill_color='rgba(200, 0, 0, 0.4)',
    stroke_color='rgba(200, 0, 0, 0.4)', scale=2
)

fig = gmaps.figure()
fig.add_layer(starbucks_layer)
fig

We can add GeoJSON to a map. This is very useful when we want to draw chloropleth maps.
import gmaps.geojson_geometries
gmaps.configure(api_key='AIza...')

countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

fig = gmaps.figure()

gini_layer = gmaps.geojson_layer(countries_geojson)
fig.add_layer(gini_layer)

## add markers, lines, etc
#Features to draw on the map
gmt_meridian = gmaps.Line(
    start=(52.0, 0.0),
    end=(50.0, 0.0),
    stroke_weight=3.0
)
greenwich = gmaps.Marker((51.3, 0.0), info_box_content='Greenwich')

drawing = gmaps.drawing_layer(features=[greenwich, gmt_meridian])
fig.add_layer(drawing)
fig

## polygons
import gmaps.datasets

london_congestion_zone_path = gmaps.datasets.load_dataset('london_congestion_zone')
london_congestion_zone_path[:2]
#[(51.530318, -0.123026), (51.530078, -0.123614)]

fig = gmaps.figure(center=(51.5, -0.1), zoom_level=12)
london_congestion_zone_polygon = gmaps.Polygon(
    london_congestion_zone_path,
    stroke_color='blue',
    fill_color='blue'
)
drawing = gmaps.drawing_layer(
    features=[london_congestion_zone_polygon],
    show_controls=False
)
fig.add_layer(drawing)

For details on how to style polygons, see the gmaps.Polygon API documentation.

See the API documentation for gmaps.drawing_layer() for an exhaustive list of options for the drawing layer.

## directions 
gmaps supports drawing routes based on the Google maps directions service. At the moment, this only supports directions between points denoted by latitude and longitude:'
import gmaps.datasets
#Latitude-longitude pairs
geneva = (46.2, 6.1)
montreux = (46.4, 6.9)
zurich = (47.4, 8.5)
fig = gmaps.figure()
geneva2zurich = gmaps.directions_layer(geneva, zurich)
fig.add_layer(geneva2zurich)

### bicycle and add stops
fig = gmaps.figure()
geneva2zurich_via_montreux = gmaps.directions_layer(
        geneva, zurich, waypoints=[montreux],
        travel_mode='BICYCLING')
fig.add_layer(geneva2zurich_via_montreux)
fig = gmaps.figure()
geneva2zurich = gmaps.directions_layer(
    geneva, zurich, show_markers=False,
    stroke_color='red', stroke_weight=3.0, stroke_opacity=1.0)
fig.add_layer(geneva2zurich)
fig''

## can add traffic

In [ ]:
heatmap_layer.point_radius = 8
heatmap_layer.max_intensity = 100
'''
heatmap.gradient = ['white', 
                    'silver', 
                    'gray']
'''
heatmap.gradient = [
    (200, 200, 200, 0.6),
    (100, 100, 100, 0.3),
    (50, 50, 50, 0.3)
]
heatmap.opacity = 0.5